In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.optim as optim
import torch.nn.functional as F

from collections import Counter, defaultdict, OrderedDict
from tqdm import tqdm
import math
import os, pickle, gc
import seaborn as sns
from scipy.stats import norm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
def log_sum_exp(A, dim=-1, keepdim=False, sum_op=torch.sum):
    """Computes `log(exp(A).sum(axis=axis))` avoiding numerical issues using the log-sum-exp trick.
    Direct calculation of :math:`\log \sum_i \exp A_i` can result in underflow or overflow numerical
    issues. Big positive values can cause overflow :math:`\exp A_i = \inf`, and big negative values
    can cause underflow :math:`\exp A_i = 0`. The latter can eventually cause the sum to go to zero
    and finally resulting in :math:`\log 0 = -\inf`.
    The log-sum-exp trick avoids these issues by using the identity,
    .. math::
        \log \sum_i \exp A_i = \log \sum_i \exp(A_i - c) + c, \text{using},  \\
        c = \max A.7
    This avoids overflow, and while underflow can still happen for individual elements it avoids
    the sum being zero.

    Parameters
    ----------
    A : tensor
        Tensor of which we wish to compute the log-sum-exp.
    axis : int, tuple, list, None
        Axis or axes to sum over; None (default) sums over all axes.
    sum_op : function
        Summing function to apply; default is torch.sum, but can also be torch.mean for log-mean-exp.

    Returns
    -------
    tensor
        The log-sum-exp of `A`, dimensions over which is summed will be dropped.
    """
    A_max = torch.max(A,dim=dim,keepdim=True)[0]
    B = torch.log(sum_op(torch.exp(A - A_max), dim=dim, keepdim=True)) + A_max
    
    if not keepdim:
        B = B.squeeze(dim)
    
    return B



class Distribution():
    def pdf(self, x):
        raise NotImplementedError

    def logpdf(self, x):
        raise NotImplementedError

    def cdf(self, x):
        raise NotImplementedError

    def logcdf(self, x):
        raise NotImplementedError

    def sample(self):
        raise NotImplementedError

    
class Normal(Distribution):
    # scalar version
    def __init__(self, loc, logvar):
        self.loc = loc
        self.logvar = logvar
        self.shp = loc.size()

        super(Normal,self).__init__()

    def logpdf(self, x, eps=0.0):
        c = - float(0.5 * math.log(2 * math.pi))
        return c - 0.5*self.logvar - (x - self.loc).pow(2) / (2 * torch.exp((self.logvar)) + eps)

    def pdf(self, x):
        return torch.exp(self.logpdf(x))

    def sample(self):
        if self.loc.is_cuda:
            eps = torch.cuda.FloatTensor(self.shp).normal_()
        else:
            eps = torch.FloatTensor(self.shp).normal_()
        return self.loc + torch.exp(0.5*self.logvar) * Variable(eps)
    
    def entropy(self):
        return 0.5 * math.log(2. * math.pi * math.e) + 0.5*self.logvar



def kl_normal2_normal2(mean1, log_var1, mean2, log_var2, eps=0.0):
    """
    Compute closed-form solution to the KL-divergence between two Gaussians parameterized
    with diagonal log variance.
    .. math::
       D_{KL}[q||p] &= -\int p(x) \log q(x) dx + \int p(x) \log p(x) dx     \\
                    &= -\int \mathcal{N}(x; \mu_2, \sigma^2_2) \log \mathcal{N}(x; \mu_1, \sigma^2_1) dx
                        + \int \mathcal{N}(x; \mu_2, \sigma^2_2) \log \mathcal{N}(x; \mu_2, \sigma^2_2) dx     \\
                    &= \frac{1}{2} \log(2\pi\sigma^2_2) + \frac{\sigma^2_1 + (\mu_1 - \mu_2)^2}{2\sigma^2_2}
                        - \frac{1}{2}( 1 + \log(2\pi\sigma^2_1) )      \\
                    &= \log \frac{\sigma_2}{\sigma_1} + \frac{\sigma^2_1 + (\mu_1 - \mu_2)^2}{2\sigma^2_2} - \frac{1}{2}
    """
    return 0.5*log_var2 - 0.5*log_var1 + (torch.exp(log_var1) + (mean1 - mean2)**2) / (2*torch.exp(log_var2) + eps) - 0.5

    
class FixedMixtureNormal(nn.Module):   #needs to be a nn.Moudle to register the parameters correcly
    # takes loc, logvar and pi as list of float values and assumes they are shared across all dimenisions
    def __init__(self, loc, logvar, pi):
        super(FixedMixtureNormal,self).__init__()
        assert sum(pi) -1 < 0.0001
        self.loc = Parameter(torch.from_numpy(np.array(loc)).float(),requires_grad=False)
        self.logvar = Parameter(torch.from_numpy(np.array(logvar)).float(),requires_grad=False)
        self.pi = Parameter(torch.from_numpy(np.array(pi)).float(),requires_grad=False)

    def _component_logpdf(self, x, eps=0.0):
        ndim = len(x.size())
        shpexpand = ndim*(None,)
        x = x.unsqueeze(-1)
        
        c = - float(0.5 * math.log(2 * math.pi))
        loc = self.loc[shpexpand]
        logvar = self.logvar[shpexpand]
        pi = self.pi[shpexpand]
        
        return c - 0.5*logvar - (x - loc).pow(2) / (2 * torch.exp(logvar) + eps)

    def logpdf(self,x):
        ndim = len(x.size())
        shpexpand = ndim*(None,)
        pi = self.pi[shpexpand]
        px = torch.exp(self._component_logpdf(x))   #... x num_components
        return torch.log(torch.sum(pi*px,-1))          

    
    
class FixedNormal(Distribution):
    # takes loc and logvar as float values and assumes they are shared across all dimenisions
    def __init__(self, loc, logvar):
        self.loc = loc
        self.logvar = logvar
        super(FixedNormal,self).__init__()

    def logpdf(self, x, eps=0.0):
        c = - float(0.5 * math.log(2 * math.pi))
        return c - 0.5*self.logvar - (x - self.loc).pow(2) / (2 * math.exp((self.logvar)) + eps)

    

In [3]:
def distribution_selector(loc, logvar, pi):    
    if isinstance(logvar,(list,tuple)) and isinstance(pi,(list,tuple)):
        assert len(logvar) == len(pi)
        num_components = len(logvar)
        print('Mixture of Normal prior, nc:',num_components)
        if not isinstance(loc,(list,tuple)):
            loc = (loc,)*num_components
        return FixedMixtureNormal(loc,logvar,pi)
    else:
        print('Normal prior')
        return FixedNormal(loc,logvar)


class BBBLinearFactorial(nn.Module):
    def __init__(self,in_features, out_features, p_logvar_init=-3, p_pi=1.0, q_logvar_init=-5):
        # p_logvar_init, p_pi can be either
        #    (list/tuples): prior model is a mixture of gaussians components=len(p_pi)=len(p_logvar_init)
        #    float: Gussian distribution
        # q_logvar_init: float, the approximate posterior is currently always a factorized gaussian
        super(BBBLinearFactorial,self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.p_logvar_init = p_logvar_init
        self.q_logvar_init = q_logvar_init
                
        #Approximate Posterior model
        self.qw_mean = Parameter(torch.Tensor(out_features, in_features))
        self.qw_logvar = Parameter(torch.Tensor(out_features, in_features))
        self.qb_mean = Parameter(torch.Tensor(out_features))
        self.qb_logvar = Parameter(torch.Tensor(out_features))     

        self.qw = Normal(loc=self.qw_mean, logvar=self.qw_logvar)
        self.qb = Normal(loc=self.qb_mean, logvar=self.qb_logvar)
            
        #Prior Model (the prior model does not have any trainable parameters so we use special versions of the normal distributions)
        self.pw = distribution_selector(loc=0.0, logvar=p_logvar_init, pi=p_pi)
        self.pb = distribution_selector(loc=0.0, logvar=p_logvar_init, pi=p_pi)

        #initialize all paramaters
        self.reset_parameters()

    def reset_parameters(self):
        #initialize (learnable) approximate posterior parameters        
        stdv = 10. / math.sqrt(self.in_features)
        self.qw_mean.data.uniform_(-stdv, stdv)
        self.qw_logvar.data.uniform_(-stdv, stdv).add_(self.q_logvar_init)
        self.qb_mean.data.uniform_(-stdv, stdv)
        self.qb_logvar.data.uniform_(-stdv, stdv).add_(self.q_logvar_init)

        
    def forward(self, input):
        raise NotImplementedError()
        
    def probforward(self, input, MAP=False):
        #input: BS, in_features
        #W: BS, in_features
        #MAP: maximum a posterior (return the mean instead of sampling from the distributions)
        if MAP:
            w_sample = self.qw.loc
            b_sample = self.qb.loc
        else:
            w_sample = self.qw.sample()
            b_sample = self.qb.sample()
        
        kl_w = torch.sum(self.qw.logpdf(w_sample) - self.pw.logpdf(w_sample))
        kl_b = torch.sum(self.qb.logpdf(b_sample) - self.pb.logpdf(b_sample))
        kl = kl_w + kl_b
        
        diagnostics = {'kl_w':kl_w.data.mean(), 'kl_b':kl_b.data.mean(), 
                       'Hq_w': self.qw.entropy().data.mean(), 'Hq_b': self.qb.entropy().data.mean()}   #Hq_w and Hq_b are the differential entropy
        output = F.linear(input, w_sample, b_sample)

        return output, kl, diagnostics


    def __repr__(self):
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'
    
    
    
class BBBMLP(nn.Module):
    def __init__(self,in_features, num_class, num_hidden, num_layers, p_logvar_init=-3, p_pi=1.0, q_logvar_init=-5):
        #create a simple MLP model with probabilistic weights
        super(BBBMLP, self).__init__()        
        layers = [BBBLinearFactorial(in_features=in_features,out_features=num_hidden, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init), nn.ELU()]
        for i in range(num_layers-1):
            layers += [BBBLinearFactorial(in_features=num_hidden,out_features=num_hidden, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init), nn.ELU()]
        layers += [BBBLinearFactorial(in_features=num_hidden,out_features=num_class, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init)]
        
        self.layers = nn.ModuleList(layers)
        self.loss = nn.CrossEntropyLoss()
        
    def probforward(self,x, MAP=False):
        diagnostics = defaultdict(list)
        kl = 0
        for l in self.layers:
            if hasattr(l, 'probforward' ) and callable( l.probforward ): 
                x, _kl, _diagnostics = l.probforward(x,MAP=MAP) 
                kl += _kl
                for k,v in _diagnostics.items():
                    diagnostics[k].append(v)
            else:
                x = l(x)        
        logits = x
        return logits, kl, diagnostics

    def getloss(self,x,y, dataset_size, MAP=False):
        logits, kl, _diagnostics = self.probforward(x, MAP=MAP)
        #_diagnostics is here a dictinary of list of floats
        # We need the dataset_size in order to 'spread' the KL divergence across all samples - this is dscribed in EQ (8) in Blundel et. al. 2015

        logpy = -self.loss(logits,y) #sample average
        kl /= dataset_size  # see EQ (8) in Blundell et al 2015

        ll = logpy - kl   #ELBO
        loss = -ll

        acc = (logits.max(1)[1].data == th_y).float().mean() #accuracy
        
        #the xxx.data.mean() is just an easy way to transfer to cpu and convert from torch to normal floats
        diagnostics = {'loss': [loss.data.mean()],
                       'll': [ll.data.mean()],
                       'kl': [kl.data.mean()],
                       'logpy': [logpy.data.mean()],
                       'acc': [acc],
                       'kl_w': _diagnostics['kl_w'],
                       'kl_b': _diagnostics['kl_b'],
                       'Hq_w': _diagnostics['Hq_w'],
                       'Hq_b': _diagnostics['Hq_b'],}
        return logits, loss, diagnostics
    
    

def plothist(model,filename):
    N = norm(loc=0,scale=np.exp(p_logvar_init))
    x =np.linspace(-0.5,0.5,100)
    W = torch.cat([model.layers[0].qw_mean.view(-1), model.layers[2].qw_mean.view(-1), model.layers[4].qw_mean.view(-1)]).data.cpu().numpy()
    b = torch.cat([model.layers[0].qb_mean.view(-1), model.layers[2].qb_mean.view(-1), model.layers[4].qb_mean.view(-1)]).data.cpu().numpy()

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    _ = plt.hist(W,np.linspace(-0.5,0.5,100),normed=True,label='q samples')
    plt.plot(x, N.pdf(x),label='prior pdf')
    plt.xlim([-0.5,0.5])
    plt.ylim([0,10])
    plt.legend()
    plt.title('Weights')

    plt.subplot(122)
    _ = plt.hist(b,np.linspace(-0.5,0.5,100),normed=True,label='q samples')
    plt.plot(x, N.pdf(x),label='prior pdf')
    plt.xlim([-0.5,0.5])
    plt.ylim([0,10])
    plt.legend()
    plt.title('bias')
    
    plt.savefig(filename)
    plt.close('all')

def addres(old,new):
    for k in new.keys():
        if k in old:
            old[k] = [vo+vn for vo,vn in zip(old[k],new[k])]
        else:
            old[k] = new[k]
    return old
    
def averres(res,num_batch):
    for k in res.keys():
        res[k] = [v/num_batch for v in res[k]]
    return res
    
def listdict2dictlist(LD):
    DL = dict(zip(sorted(LD[0].keys()),zip(*[[v for k,v in sorted(d.items())]for d in LD])))
    return DL
    

In [4]:
data = np.load('mnist.npz')  #Download from https://www.dropbox.com/s/k92825vinroxh6i/mnist.npz?dl=0
xtrain, ytrain = data['x_train'],data['y_train']
xvalid, yvalid = data['x_valid'],data['y_valid']
xtest, ytest = data['x_test'],data['y_test']


#MU, SIGMA = xtrain.mean(keepdims=True), xtrain.std(keepdims=True)
#xtrain = (xtrain-0.5)/0.5
#xvalid = (xvalid-0.5)/0.5
#xtest = (xtest-0.5)/0.5


In [5]:
batch_size = 128
nsamples = 16
dataset_size, in_features = xtrain.shape
num_hidden = 512
num_layers = 2
p_logvar_init = -5 #[0.,-1,-3,-5,-7]
q_logvar_init = -5 #[-4,-5,-6]
p_pi = 1.0 #(1./10.,2./10.,4./10.,2./10.,1./10.)

num_class = 10
learningrate = 1e-3
num_epochs = 100
cuda = True
save_every_epoch = 1

for p_logvar_init in [0.,-1,-3,-5,-7]:
    for q_logvar_init in [-4,-5,-6]:   

        expname = 'normal_prior_qlogvar%0.1f_plogvar%0.1f_lr%0.5f_bs%i_ns%i'%(q_logvar_init,p_logvar_init,learningrate,batch_size,nsamples)
        weights_dir = os.path.join(expname,'weights')
        figure_dir = os.path.join(expname,'figures')

        logfile = os.path.join(expname,'logfile.txt')
        diagnosticsfile = os.path.join(expname,'diagnostics.pkl')
        weightsfile = os.path.join(weights_dir,'model_epoch%i.pkl')
        histfigurefile = os.path.join(figure_dir,'weighthistogram_epoch%i.png')


        if not os.path.exists(expname): os.makedirs(expname)
        if not os.path.exists(weights_dir): os.makedirs(weights_dir)  
        if not os.path.exists(figure_dir): os.makedirs(figure_dir)  
        with open(logfile,'w') as fh: fh.write('')

        model = BBBMLP(in_features=in_features, num_class=num_class, num_hidden=num_hidden, num_layers=num_layers, p_logvar_init=p_logvar_init, p_pi=p_pi, q_logvar_init=q_logvar_init)
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learningrate)
        th_x = torch.FloatTensor(batch_size*nsamples,in_features).fill_(0)
        th_y = torch.LongTensor(batch_size*nsamples).fill_(0)

        if cuda:
            model.cuda()
            th_x = th_x.cuda()
            th_y = th_y.cuda()


        def run_epoch(np_x, np_y, batch_size, dataset_size, MAP=False, is_training=False):
            N = np_x.shape[0]
            idx = np.arange(N)
            np.random.shuffle(idx)
            diagnostics = {}
            nbatch_per_epoch = N//batch_size

            for i in tqdm(range(nbatch_per_epoch)):
                batch_idx = idx[i*batch_size : (i+1)*batch_size]

                th_x.copy_(torch.from_numpy(np.tile(np_x[batch_idx],[nsamples,1])))
                th_y.copy_(torch.from_numpy(np.tile(np_y[batch_idx],[nsamples])))

                logits, loss, _diagnostics = model.getloss(Variable(th_x),Variable(th_y), dataset_size=dataset_size, MAP=MAP)
                diagnostics = addres(diagnostics, _diagnostics)
                if is_training:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
            diagnostics = averres(diagnostics,nbatch_per_epoch)
            return diagnostics



        diagnostics_batch_train, diagnostics_batch_valid, diagnostics_batch_valid_MAP = [],[],[]
        for e in range(num_epochs):

            if e % save_every_epoch == 0:
                #get weights before training
                weights = model.state_dict()
                for k in weights: 
                    weights[k] = weights[k].cpu()
                plothist(model,histfigurefile%(e))   

            diagnostics_batch_train += [run_epoch(xtrain, ytrain, batch_size, dataset_size, MAP=False, is_training=True)]
            diagnostics_batch_valid += [run_epoch(xvalid, yvalid, batch_size, dataset_size, MAP=False, is_training=False)]
            diagnostics_batch_valid_MAP += [run_epoch(xvalid, yvalid, batch_size, dataset_size, MAP=True, is_training=False)]

            ltr = "Train %i | "%(e) + ", ".join(["%s: %s"%(k,"|".join(["%0.3f"%(_v) for _v in v])) for k,v in sorted(diagnostics_batch_train[-1].items())])
            lte = "Test %i | "%(e)     + ", ".join(["%s: %s"%(k,"|".join(["%0.3f"%(_v) for _v in v])) for k,v in sorted(diagnostics_batch_valid[-1].items())])
            ltemap = "MAP-Test %i | "%(e)     + ", ".join(["%s: %s"%(k,"|".join(["%0.3f"%(_v) for _v in v])) for k,v in sorted(diagnostics_batch_valid_MAP[-1].items())])

            print('\n'.join([ltr,lte,ltemap]))
            with open(logfile,'a') as fh: fh.write('\n'.join([ltr,lte,ltemap]) + '\n')

            if e % save_every_epoch == 0:
                with open(diagnosticsfile,'wb') as fh: 
                    pickle.dump({'train':listdict2dictlist(diagnostics_batch_train),
                                 'valid':listdict2dictlist(diagnostics_batch_valid),
                                 'validMAP':listdict2dictlist(diagnostics_batch_valid_MAP)},fh)
                with open(weightsfile%(e),'wb') as fh: pickle.dump(weights,fh)
            gc.collect()
          

Normal prior
Normal prior
Normal prior
Normal prior
Normal prior
Normal prior


100%|██████████| 78/78 [00:00<00:00, 90.16it/s]


Train 0 | Hq_b: -0.584|-0.578|-0.570, Hq_w: -0.558|-0.581|-0.583, acc: 0.717, kl: 20.362, kl_b: 785.552|788.714|15.265, kl_w: 604334.179|404266.976|7897.876, ll: -31.343, logpy: -10.981, loss: 31.343
Test 0 | Hq_b: -0.585|-0.578|-0.570, Hq_w: -0.536|-0.581|-0.585, acc: 0.843, kl: 20.177, kl_b: 785.325|790.718|15.344, kl_w: 595220.730|404133.183|7917.562, ll: -24.212, logpy: -4.035, loss: 24.212
MAP-Test 0 | Hq_b: -0.585|-0.578|-0.570, Hq_w: -0.536|-0.581|-0.585, acc: 0.922, kl: 26.742, kl_b: 1036.586|1039.103|20.190, kl_w: 791808.000|532750.000|10421.385, ll: -28.504, logpy: -1.762, loss: 28.504


100%|██████████| 78/78 [00:00<00:00, 89.92it/s]


Train 1 | Hq_b: -0.585|-0.577|-0.571, Hq_w: -0.514|-0.580|-0.587, acc: 0.842, kl: 20.003, kl_b: 786.075|788.194|15.357, kl_w: 586600.789|404038.599|7910.718, ll: -23.576, logpy: -3.573, loss: 23.576
Test 1 | Hq_b: -0.586|-0.577|-0.572, Hq_w: -0.492|-0.580|-0.588, acc: 0.864, kl: 19.825, kl_b: 787.846|787.787|15.179, kl_w: 577863.279|403868.620|7921.231, ll: -22.379, logpy: -2.554, loss: 22.379
MAP-Test 1 | Hq_b: -0.586|-0.577|-0.572, Hq_w: -0.492|-0.580|-0.588, acc: 0.937, kl: 26.378, kl_b: 1037.294|1038.572|20.198, kl_w: 773898.000|532474.875|10433.107, ll: -27.246, logpy: -0.868, loss: 27.246


...

Train 22 | Hq_b: -0.981|-0.622|-0.988, Hq_w: 0.101|-0.650|-1.071, acc: 0.979, kl: 16.593, kl_b: 986.356|808.254|19.341, kl_w: 396221.788|421249.971|10341.068, ll: -16.720, logpy: -0.127, loss: 16.720
Test 22 | Hq_b: -0.976|-0.605|-0.987, Hq_w: 0.114|-0.635|-1.070, acc: 0.966, kl: 16.424, kl_b: 982.212|798.694|19.269, kl_w: 391740.630|417339.827|10341.730, ll: -16.755, logpy: -0.331, loss: 16.755
MAP-Test 22 | Hq_b: -0.976|-0.605|-0.987, Hq_w: 0.114|-0.635|-1.070, acc: 0.975, kl: 21.810, kl_b: 1237.855|1050.558|24.288, kl_w: 529242.812|546075.875|12877.807, ll: -22.029, logpy: -0.219, loss: 22.029


100%|██████████| 78/78 [00:00<00:00, 89.63it/s]


Train 23 | Hq_b: -0.971|-0.589|-0.981, Hq_w: 0.127|-0.620|-1.069, acc: 0.980, kl: 16.260, kl_b: 981.551|789.964|19.235, kl_w: 387406.048|413491.272|10331.227, ll: -16.384, logpy: -0.124, loss: 16.384
Test 23 | Hq_b: -0.966|-0.573|-0.977, Hq_w: 0.139|-0.605|-1.068, acc: 0.965, kl: 16.096, kl_b: 980.046|782.831|19.521, kl_w: 383126.297|409569.740|10317.752, ll: -16.499, logpy: -0.403, loss: 16.499
MAP-Test 23 | Hq_b: -0.966|-0.573|-0.977, Hq_w: 0.139|-0.605|-1.068, acc: 0.975, kl: 21.445, kl_b: 1232.882|1033.692|24.183, kl_w: 518960.594|538154.250|12866.468, ll: -21.682, logpy: -0.237, loss: 21.682


 96%|█████████▌| 373/390 [00:08<00:00, 41.86it/s]


KeyboardInterrupt: 